# Data inspection - comments data set in BigQuery

One criteria I've chosen to filter my data is count the number of comments for each subreddit during all years and consider the subreddits with more than 500.000 comments as a target of most famouse subreddit.

Before reach this conclusion I've needed to explore my data understanding the best criteria.

Here below the query the steps followed.

First of all we need the authentication

In [2]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


Enable data table display




In [0]:
%load_ext google.colab.data_table

In [5]:
%env GCLOUD_PROJECT=reddit-254019

env: GCLOUD_PROJECT=reddit-254019


I decided to explore only one year (2018) filtering the data by:

*   subreddit
*   count of of comments (descending)
*   year
*   moth

grouping by subreddit, year and month and ordering by number of comments

In [4]:
%%bigquery --project reddit-254019
SELECT
  subreddit,
  COUNT(body) AS number_comments,
  EXTRACT(YEAR
  FROM
    TIMESTAMP_SECONDS(created_utc)) year,
  EXTRACT(month
  FROM
    TIMESTAMP_SECONDS(created_utc)) month
FROM
  `fh-bigquery.reddit_comments.2018_*` 
GROUP BY
  1,
  3,
  4
ORDER BY
  number_comments DESC

,subreddit,number_comments,year,month
0,AskReddit,6003147,2018,12
1,AskReddit,5834042,2018,10
2,AskReddit,5765531,2018,8
3,AskReddit,5652413,2018,7
4,AskReddit,5615448,2018,11
...,...,...,...,...
1294890,u_robotseth,1,2018,1
1294891,Billiepiper,1,2018,1
1294892,musiciansofthefence,1,2018,1
1294893,UnityAssetMall,1,2018,1


As this result is not completely helpfull, I decided to nest my current query with another that showes me subreddits not repeated and with a limit of 1.000.000 count of comments (arbitrary alection)

In [5]:
%%bigquery --project reddit-254019
FROM (
  SELECT
    subreddit,
    COUNT(subreddit) AS present_month
  FROM (
    SELECT
      subreddit,
      COUNT(body) AS number_comments,
      EXTRACT(YEAR
      FROM
        TIMESTAMP_SECONDS(created_utc)) year,
      EXTRACT(month
      FROM
        TIMESTAMP_SECONDS(created_utc)) month
    FROM
      `fh-bigquery.reddit_comments.2014`
    GROUP BY
      1,
      3,
      4
    ORDER BY
      number_comments DESC)
  WHERE
    number_comments > 500000
  GROUP BY
    subreddit
  ORDER BY
    subreddit ASC 

Executing query with job ID: e1ec32ea-9552-4a3c-b765-706c6b89d051
Query executing: 0.26s

BadRequest: ignored

This query explain better the data. 
I've decided to discart the subreddits with more than 1.000.000 comments but with only a few months with this criteria.

My new query will consider instead:



*   500.000 as count of numner of comments
*   the subreddits where the fist cirteria as been followed during 12 months in a year

These new filters gives me a better sample of the trend of the most "famouse"



In [0]:
%%bigquery --project reddit-254019
SELECT
  *
FROM (
  SELECT
    subreddit,
    COUNT(subreddit) AS present_month
  FROM (
    SELECT
      subreddit,
      COUNT(body) AS number_comments,
      EXTRACT(YEAR
      FROM
        TIMESTAMP_SECONDS(created_utc)) year,
      EXTRACT(month
      FROM
        TIMESTAMP_SECONDS(created_utc)) month
    FROM
      `fh-bigquery.reddit_comments.2014`
    GROUP BY
      1,
      3,
      4
    ORDER BY
      number_comments DESC)
  WHERE
    number_comments > 500000
  GROUP BY
    subreddit
  ORDER BY
    subreddit ASC )
WHERE
  present_month = 12